In [3]:
import cv2

def split_video(input_video, output_folder):
    # 打开视频文件
    cap = cv2.VideoCapture(input_video)

    # 获取视频的帧率
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # 检查视频是否成功打开
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # 设置帧计数器
    frame_count = 0

    # 读取视频帧
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # 生成输出文件名
        output_filename = f"{output_folder}/frame_{frame_count:03d}.jpg"

        # 保存当前帧为图像文件
        cv2.imwrite(output_filename, frame)

        # 打印处理进度
        print(f"Processed frame {frame_count}")

        # 增加帧计数器
        frame_count += 1

    # 释放资源
    cap.release()

    print("Video processing complete.")

# 示例用法
input_video_path = "ex_2_video.mp4"
output_folder_path = "./split"
split_video(input_video_path, output_folder_path)


24
Processed frame 0
Processed frame 1
Processed frame 2
Processed frame 3
Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25
Processed frame 26
Processed frame 27
Processed frame 28
Processed frame 29
Processed frame 30
Processed frame 31
Processed frame 32
Processed frame 33
Processed frame 34
Processed frame 35
Processed frame 36
Processed frame 37
Processed frame 38
Processed frame 39
Processed frame 40
Processed frame 41
Processed frame 42
Processed frame 43
Processed frame 44
Processed frame 45
Processed frame 46
Processed frame 47
Processed frame 48
Processed frame 49
Processed frame 50
Processed frame 51
Processed frame 52


In [6]:
import cv2
import os
import numpy as np
import math
import houghlines
import draw_lines

def generate_gaussian_filter(sigma, size=3):
    kernel_size = int(size)
    kernel = np.fromfunction(lambda x, y: (1/ (2 * np.pi * sigma ** 2)) * np.exp(- ((x - (kernel_size - 1) / 2) ** 2 + (y - (kernel_size - 1) / 2) ** 2) / (2 * sigma ** 2)), (kernel_size, kernel_size))
    return kernel / np.sum(kernel) 


def conv2d(img, kernel, bias = 0, stride=1, padding=0):
    input_height, input_width = img.shape
    kernel_height, kernel_width = kernel.shape

    output_height = (input_height - kernel_height + 2 * padding) // stride + 1
    output_width = (input_width - kernel_width + 2 * padding) // stride + 1

    if padding > 0:
        padded_input = np.pad(img, pad_width=padding, mode='constant', constant_values=0)
    else:
        padded_input = img

    output = np.zeros((output_height, output_width))

    for i in range(0, input_height - kernel_height + 1, stride):
        for j in range(0, input_width - kernel_width + 1, stride):
            window = padded_input[i:i + kernel_height, j:j + kernel_width]
            output[i // stride, j // stride] = np.sum(window * kernel) + bias

    return output


def canny_edge_detection(image, low_threshold, high_threshold):
    #计算梯度
    sobel_x = conv2d(image, np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]), 0, 1, 0)
    sobel_y = conv2d(image, np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]]), 0, 1, 0)
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    gradient_direction = np.arctan2(sobel_y, sobel_x)

    #非极大值抑制
    suppressed_magnitude = non_maximum_suppression(gradient_magnitude, gradient_direction)

    #双阈值边缘跟踪
    edges = edge_tracking(suppressed_magnitude, low_threshold, high_threshold)

    return edges.astype(np.uint8)


def non_maximum_suppression(magnitude, direction):
    height, width = magnitude.shape
    suppressed_magnitude = np.zeros_like(magnitude)

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            angle = direction[i, j]

            q = 255
            r = 255

            # 竖直边缘
            if (0 <= angle < np.pi / 8) or (7 * np.pi / 8 <= angle <= np.pi):
                q = magnitude[i, j+1]
                r = magnitude[i, j-1]
            # 45度边缘
            elif (np.pi / 8 <= angle < 3 * np.pi / 8):
                q = magnitude[i+1, j-1]
                r = magnitude[i-1, j+1]
            # 水平边缘
            elif (3 * np.pi / 8 <= angle < 5 * np.pi / 8):
                q = magnitude[i+1, j]
                r = magnitude[i-1, j]
            # 135度边缘
            elif (5 * np.pi / 8 <= angle < 7 * np.pi / 8):
                q = magnitude[i-1, j-1]
                r = magnitude[i+1, j+1]

            if magnitude[i, j] >= q and magnitude[i, j] >= r:
                suppressed_magnitude[i, j] = magnitude[i, j]

    return suppressed_magnitude

def edge_tracking(magnitude, low_threshold, high_threshold):
    edges = np.zeros_like(magnitude)

    strong_edges = magnitude > high_threshold
    weak_edges = (magnitude >= low_threshold) & (magnitude <= high_threshold)

    edges[strong_edges] = 255
    edges[weak_edges] = 50  # 需要自己根据情况调整阈值

    # 通过连接弱边缘，形成连通的强边缘
    for i in range(1, magnitude.shape[0] - 1):
        for j in range(1, magnitude.shape[1] - 1):
            if edges[i, j] == 50:
                if np.any(edges[i-1:i+2, j-1:j+2] == 255):
                    edges[i, j] = 255

    return edges


frames = [f for f in os.listdir("./split") if f.endswith(".jpg")]

for img_name in frames:
    img = cv2.imread(os.path.join("./split/" + img_name))
    print("Process:" + img_name)
    
    #灰度转换
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #高斯滤波
    gray = conv2d(gray, generate_gaussian_filter(1.5), 0, 1, 0)
    
    #使用Canny的边缘检测 Canny要求输入图像的深度8位无符号整数 暂时调用库实现
    gray = np.uint8(gray)
    edges = canny_edge_detection(gray, 50, 150)
    #edges = cv2.Danny(gray, 50, 150)
    #生成mask掩膜，提取感兴趣区域ROI
    height, width = gray.shape
    mask = np.zeros_like(gray)
    # 定义ROI区域
    vertices = np.array([[(80, height), (width / 2, height / 2 + 15), (width - 30, height)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, 255)
    mask = cv2.resize(mask, (edges.shape[1], edges.shape[0]))
    edges = cv2.bitwise_and(edges, mask)
    
    #霍夫变换
    lines = houghlines.Hough_Line(edges, img)
    #lines = hough_transform(edges)
    #lines = lines.reshape(-1, 1, 4)
    #lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=100, maxLineGap=50)
    
    #在原始图像上绘制车道线
    draw_lines.draw_lines(img, lines)
    '''
    for line in lines:
        x1, y1 = line[0]
        x2, y2 = line[1]
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    '''
    
    cv2.imwrite(os.path.join("./afterwork/" + img_name), img)
    break
    
    

Process:frame_000.jpg


In [9]:
import cv2
import os

def compile_frames(input_folder, output_video, fps):
    # 获取图像文件夹中的所有文件
    frames = [f for f in os.listdir(input_folder) if f.endswith(".jpg")]

    # 按文件名排序确保顺序正确
    frames.sort()

    # 获取第一帧的尺寸
    frame = cv2.imread(os.path.join(input_folder, frames[0]))
    height, width, _ = frame.shape

    # 创建视频编写器
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    # 逐帧写入视频
    for frame_name in frames:
        frame_path = os.path.join(input_folder, frame_name)
        frame = cv2.imread(frame_path)

        # 写入视频
        out.write(frame)

    # 释放资源
    out.release()

    print("Video compilation complete.")

# 示例用法
input_folder_path = "./afterwork"
output_video_path = "after_work.mp4"
frame_rate = 24

compile_frames(input_folder_path, output_video_path, frame_rate)


Video compilation complete.
